# Création d'un dictionnaire qui contient un tableau pour chaque ligne de métro

## Chaque tableau affiche les stations de la ligne, leur fréquentaiton annuelle, leur fréquentation hebdomadaire moyenne sur la période 24/07/2022-11/08/2022 (période des JO mais en 2022)

In [1]:
import pandas as pd
import numpy as np
import re
import difflib
from IPython.utils import io

Importer le notebook Nettoyage données affluence par jour + nombre de validation par jour par station

In [2]:
try:
    with io.capture_output():
        %run -i "Nettoyage données affluence par jour + nombre de validation par jour par station.ipynb"
except Exception as e:
    display(e)

Importer les données disponibles sur le site https://data.ratp.fr .
Ce jeu de données présente le trafic des entrants directs sur le réseau ferré RATP en 2021

In [3]:
%pip install openpyxl
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print("Tableau importé : trafic des entrants directs sur le réseau ferré RATP en 2021")
display(tableau_fréquentation)


Note: you may need to restart the kernel to use updated packages.


,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,3,Métro,GARE DE LYON,28640475,1,14,A,NaN,NaN,Paris,12.0
1,12,Métro,BELLEVILLE,7314438,2,11,NaN,NaN,NaN,Paris,10.0
2,32,Métro,OPERA,5193831,3,7,8,NaN,NaN,Paris,9.0
3,58,Métro,SAINT-MANDE-TOURELLE,3944640,1,NaN,NaN,NaN,NaN,Saint Mandé,NaN
4,59,Métro,HOCHE,3928404,5,NaN,NaN,NaN,NaN,Pantin,NaN
...,...,...,...,...,...,...,...,...,...,...,...
366,39,RER,FONTENAY-SOUS-BOIS,2121720,A,NaN,NaN,NaN,NaN,Fontenay-sous-Bois,NaN
367,40,RER,SUCY-BONNEUIL,1993503,A,NaN,NaN,NaN,NaN,Sucy-en-Brie,NaN
368,58,RER,LE GUICHET,530706,B,NaN,NaN,NaN,NaN,Orsay,NaN
369,65,RER,PARC DE SCEAUX,383626,B,NaN,NaN,NaN,NaN,Antony,NaN


Nettoyage du tableau

In [4]:
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_fréquentation = tableau_fréquentation.drop_duplicates(subset='Station')

colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5
0,3,GARE DE LYON,28640475,1,14,,nan,nan
1,12,BELLEVILLE,7314438,2,11,nan,nan,nan
2,32,OPERA,5193831,3,7,8,nan,nan
3,58,SAINT-MANDE-TOURELLE,3944640,1,nan,nan,nan,nan
4,59,HOCHE,3928404,5,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...
350,272,SULLY-MORLAND,1124169,7,nan,nan,nan,nan
351,275,EUROPE,1106459,3,nan,nan,nan,nan
352,283,CAMPO-FORMIO,1016150,5,nan,nan,nan,nan
353,298,SAINT-FARGEAU,472258,3bis,nan,nan,nan,nan


In [5]:
# Ajouter la colonne 'nb_correspondance'
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].apply(lambda row: sum(pd.notna(value) and value != 'nan' for value in row), axis=1)
display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance
0,3,GARE DE LYON,28640475,1,14,,nan,nan,3
1,12,BELLEVILLE,7314438,2,11,nan,nan,nan,2
2,32,OPERA,5193831,3,7,8,nan,nan,3
3,58,SAINT-MANDE-TOURELLE,3944640,1,nan,nan,nan,nan,1
4,59,HOCHE,3928404,5,nan,nan,nan,nan,1
...,...,...,...,...,...,...,...,...,...
350,272,SULLY-MORLAND,1124169,7,nan,nan,nan,nan,1
351,275,EUROPE,1106459,3,nan,nan,nan,nan,1
352,283,CAMPO-FORMIO,1016150,5,nan,nan,nan,nan,1
353,298,SAINT-FARGEAU,472258,3bis,nan,nan,nan,nan,1


In [6]:
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
display(tableau_fréquentation)

,Rang,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nb_correspondance
0,3,GARE DE LYON,9546825.0,1,14,,nan,nan,3
1,12,BELLEVILLE,3657219.0,2,11,nan,nan,nan,2
2,32,OPERA,1731277.0,3,7,8,nan,nan,3
3,58,SAINT-MANDE-TOURELLE,3944640.0,1,nan,nan,nan,nan,1
4,59,HOCHE,3928404.0,5,nan,nan,nan,nan,1
...,...,...,...,...,...,...,...,...,...
350,272,SULLY-MORLAND,1124169.0,7,nan,nan,nan,nan,1
351,275,EUROPE,1106459.0,3,nan,nan,nan,nan,1
352,283,CAMPO-FORMIO,1016150.0,5,nan,nan,nan,nan,1
353,298,SAINT-FARGEAU,472258.0,3bis,nan,nan,nan,nan,1


Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation par jour à chaque station

In [7]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.315789
1,ablon,1101.105263
2,acheres grand,51.388889
3,acheres ville,3012.052632
4,aeroport cdg1,5050.947368
...,...,...
733,vosves,14.894737
734,vulaines sur se,42.894737
735,wagram,3268.157895
736,yerres,2743.000000


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne 

In [8]:
from IPython.display import display

numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in str(x.values), axis=1)]
    
    # Supprimer les doublons dans la colonne 'Station'
    filtered_rows = filtered_rows.drop_duplicates(subset='Station')
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')
    tableaux[tableau_nom] = tableau.drop_duplicates(subset='Station')  # Ajoutez cette ligne pour supprimer les doublons de la colonne 'Station'


In [9]:

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [10]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [11]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [12]:

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les doublons de stations
    current_table = current_table.drop_duplicates(subset='Station')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table


In [13]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

In [14]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


In [15]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic',
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne',
    'fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")
print(tableau.columns)
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")

Colonnes disponibles :
Index(['Rang', 'Station', 'Trafic', 'Correspondance_1', 'Correspondance_2',
       'Correspondance_3', 'Correspondance_4', 'Correspondance_5',
       'nb_correspondance', 'fréquentation_totale_ligne',
       'Part du trafic de la station par rapport à toute la ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
       '00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00',
       '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00',
       '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00',
       '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00',
       '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00',
       '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-00:00',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction',
       'nombre_validations_totales_pour_la_ligne'],
      dtype='object')
Tableau: ligne numéro[1]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.031973,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,belleville,3657219.0,0.012248,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
3,saint-mande-tourelle,3944640.0,0.013211,8727.684211,8727.684211,4363.842105,1,1,nan,nan,...,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99,58
4,bercy,1942106.0,0.006504,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
6,saint-denis-universite,3569990.0,0.011956,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,michel-ange-auteuil,756025.0,0.002532,2899.631579,1449.815789,724.907895,2,9,10,nan,...,37.63,40.86,43.35,40.46,35.29,21.50,11.78,11.57,9.71,234
124,notre-dame-des-champs,1487256.0,0.004981,1661.578947,1661.578947,830.789474,1,12,nan,nan,...,36.70,46.97,51.64,51.20,38.02,25.48,14.76,14.98,12.61,238
125,malakoff-rue etienne dolet,1450451.0,0.004858,3242.105263,3242.105263,1621.052632,1,13,nan,nan,...,33.51,34.59,43.10,41.24,28.00,15.65,8.94,6.14,4.88,239
126,michel-ange-molitor,710276.0,0.002379,2581.315789,1290.657895,645.328947,2,9,10,nan,...,37.02,39.89,42.74,38.67,35.84,19.83,11.89,10.72,13.92,243




Tableau: ligne numéro[2]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,belleville,3.657219e+06,0.031606,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
1,colonel fabien,3.043606e+06,0.026303,6090.315789,6090.315789,3045.157895,1,2,nan,nan,...,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86,104
2,menilmontant,2.847264e+06,0.024606,5824.315789,5824.315789,2912.157895,1,2,nan,nan,...,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79,118
3,blanche,2.167570e+06,0.018732,4236.421053,4236.421053,2118.210526,1,2,nan,nan,...,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68,163
4,rue du bac,1.423364e+06,0.012301,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
5,philippe auguste,1.262653e+06,0.010912,2473.842105,2473.842105,1236.921053,1,2,nan,nan,...,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82,256
6,monceau,1.148677e+06,0.009927,2529.789474,2529.789474,1264.894737,1,2,nan,nan,...,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00,268
7,rennes,8.613340e+05,0.007444,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
8,montparnasse-bienvenue,5.101806e+06,0.044090,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
10,madeleine,1.776976e+06,0.015357,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27




Tableau: ligne numéro[3]


...

,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,rue du bac,1.423364e+06,0.023459,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
1,rennes,8.613340e+05,0.014196,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
2,montparnasse-bienvenue,5.101806e+06,0.084085,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
4,madeleine,1.776976e+06,0.029287,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,sevres-babylone,1.696252e+06,0.027957,6540.105263,3270.052632,1635.026316,2,10,12,nan,...,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96,88
6,mairie d'issy,2.874138e+06,0.047370,6616.526316,6616.526316,3308.263158,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
7,notre-dame-de-lorette,1.893828e+06,0.031213,3949.578947,3949.578947,1974.789474,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188
8,porte de versailles,3.268157e+06,0.053864,15023.947368,15023.947368,7511.973684,1,12,nan,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,93
9,vaugirard,2.482886e+06,0.040921,4768.736842,4768.736842,2384.368421,1,12,nan,nan,...,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66,136
10,marx dormoy,2.425928e+06,0.039983,5344.315789,5344.315789,2672.157895,1,12,nan,nan,...,36.82,37.68,43.01,38.88,31.06,21.78,14.49,10.47,8.05,139




Tableau: ligne numéro[13]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-denis-universite,3569990.0,0.042992,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
1,les courtilles,2802689.0,0.033751,8152.789474,8152.789474,4076.394737,1,13,nan,nan,...,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88,122
2,porte de saint-ouen,2710638.0,0.032643,6262.368421,6262.368421,3131.184211,1,13,nan,nan,...,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27,125
3,montparnasse-bienvenue,5101806.0,0.061438,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
5,porte de clichy,2639248.5,0.031783,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,place clichy,2580966.0,0.031081,10922.736842,5461.368421,2730.684211,2,2,13,nan,...,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26,33
7,porte de vanves,3395358.0,0.040889,7711.315789,7711.315789,3855.657895,1,13,nan,nan,...,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11,87
8,pernety,2173567.0,0.026175,4404.263158,4404.263158,2202.131579,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
9,carrefour pleyel,1535528.0,0.018492,3933.789474,3933.789474,1966.894737,1,13,nan,nan,...,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78,230
10,gabriel peri,3790573.0,0.045648,9837.842105,9837.842105,4918.921053,1,13,nan,nan,...,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40,64




Tableau: ligne numéro[14]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.172042,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,bercy,1942106.0,0.034998,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
4,madeleine,1776976.0,0.032023,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,porte de clichy,2639248.5,0.047561,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,cour saint-emilion,2985122.0,0.053794,3173.444444,3173.444444,1586.722222,1,14,nan,nan,...,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76,110
7,bibliotheque,11104474.0,0.200112,6088.000000,6088.000000,3044.000000,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
8,chatelet,1670158.8,0.030098,87779.526316,17555.905263,8777.952632,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
9,olympiades,5214595.0,0.093971,12997.142857,12997.142857,6498.571429,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
10,saint-lazare,6625676.8,0.119400,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
12,mairie de saint-ouen,2415405.0,0.043528,12017.736842,6008.868421,3004.434211,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39


In [16]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic', 
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne','fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")


Colonnes disponibles :


In [17]:
# Nouveaux noms des colonnes
nouveaux_noms_colonnes = {
    'Station': 'Station',
    'Trafic': 'Trafic annuel dans la station',
    'Part du trafic de la station par rapport à toute la ligne': 'Part de la station dans le trafic annuel de la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station': 'Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO',
    'Correspondance_1': 'Correspondance 1',
    'Correspondance_2': 'Correspondance 2',
    'Correspondance_3': 'Correspondance 3',
    'Correspondance_4': 'Correspondance 4',
    'Correspondance_5': 'Correspondance 5',
    'nb_correspondance': 'Nombre de correspondances de la station',
    'nombre_validations_totales_pour_la_ligne': 'Nombre de validations annuelles sur la ligne',
    'fréquentation_totale_ligne': 'Fréquentation totale de la ligne',
    '00:00-01:00': '00:00-01:00',
    '01:00-02:00': '01:00-02:00',
    '02:00-03:00': '02:00-03:00',
    '03:00-04:00': '03:00-04:00',
    '04:00-05:00': '04:00-05:00',
    '05:00-06:00': '05:00-06:00',
    '06:00-07:00': '06:00-07:00',
    '07:00-08:00': '07:00-08:00',
    '08:00-09:00': '08:00-09:00',
    '09:00-10:00': '09:00-10:00',
    '10:00-11:00': '10:00-11:00',
    '11:00-12:00': '11:00-12:00',
    '12:00-13:00': '12:00-13:00',
    '13:00-14:00': '13:00-14:00',
    '14:00-15:00': '14:00-15:00',
    '15:00-16:00': '15:00-16:00',
    '16:00-17:00': '16:00-17:00',
    '17:00-18:00': '17:00-18:00',
    '18:00-19:00': '18:00-19:00',
    '19:00-20:00': '19:00-20:00',
    '20:00-21:00': '20:00-21:00',
    '21:00-22:00': '21:00-22:00',
    '22:00-23:00': '22:00-23:00',
    '23:00-00:00': '23:00-00:00',
    'Rang': 'Rang'
}

# Afficher chaque tableau final avec le nouvel ordre des colonnes et les nouveaux noms
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    # Renommer les colonnes
    tableau = tableau.rename(columns=nouveaux_noms_colonnes)
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")


Tableau: ligne numéro[1]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.031973,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,belleville,3657219.0,0.012248,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
3,saint-mande-tourelle,3944640.0,0.013211,8727.684211,8727.684211,4363.842105,1,1,nan,nan,...,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99,58
4,bercy,1942106.0,0.006504,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
6,saint-denis-universite,3569990.0,0.011956,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,michel-ange-auteuil,756025.0,0.002532,2899.631579,1449.815789,724.907895,2,9,10,nan,...,37.63,40.86,43.35,40.46,35.29,21.50,11.78,11.57,9.71,234
124,notre-dame-des-champs,1487256.0,0.004981,1661.578947,1661.578947,830.789474,1,12,nan,nan,...,36.70,46.97,51.64,51.20,38.02,25.48,14.76,14.98,12.61,238
125,malakoff-rue etienne dolet,1450451.0,0.004858,3242.105263,3242.105263,1621.052632,1,13,nan,nan,...,33.51,34.59,43.10,41.24,28.00,15.65,8.94,6.14,4.88,239
126,michel-ange-molitor,710276.0,0.002379,2581.315789,1290.657895,645.328947,2,9,10,nan,...,37.02,39.89,42.74,38.67,35.84,19.83,11.89,10.72,13.92,243




Tableau: ligne numéro[2]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,belleville,3.657219e+06,0.031606,16611.631579,8305.815789,4152.907895,2,2,11,nan,...,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97,12
1,colonel fabien,3.043606e+06,0.026303,6090.315789,6090.315789,3045.157895,1,2,nan,nan,...,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86,104
2,menilmontant,2.847264e+06,0.024606,5824.315789,5824.315789,2912.157895,1,2,nan,nan,...,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79,118
3,blanche,2.167570e+06,0.018732,4236.421053,4236.421053,2118.210526,1,2,nan,nan,...,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68,163
4,rue du bac,1.423364e+06,0.012301,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
5,philippe auguste,1.262653e+06,0.010912,2473.842105,2473.842105,1236.921053,1,2,nan,nan,...,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82,256
6,monceau,1.148677e+06,0.009927,2529.789474,2529.789474,1264.894737,1,2,nan,nan,...,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00,268
7,rennes,8.613340e+05,0.007444,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
8,montparnasse-bienvenue,5.101806e+06,0.044090,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
10,madeleine,1.776976e+06,0.015357,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27




Tableau: ligne numéro[3]


...

,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,rue du bac,1.423364e+06,0.023459,2642.842105,2642.842105,1321.421053,1,12,nan,nan,...,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93,242
1,rennes,8.613340e+05,0.014196,1546.052632,1546.052632,773.026316,1,12,nan,nan,...,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72,288
2,montparnasse-bienvenue,5.101806e+06,0.084085,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
4,madeleine,1.776976e+06,0.029287,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,sevres-babylone,1.696252e+06,0.027957,6540.105263,3270.052632,1635.026316,2,10,12,nan,...,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96,88
6,mairie d'issy,2.874138e+06,0.047370,6616.526316,6616.526316,3308.263158,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
7,notre-dame-de-lorette,1.893828e+06,0.031213,3949.578947,3949.578947,1974.789474,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188
8,porte de versailles,3.268157e+06,0.053864,15023.947368,15023.947368,7511.973684,1,12,nan,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,93
9,vaugirard,2.482886e+06,0.040921,4768.736842,4768.736842,2384.368421,1,12,nan,nan,...,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66,136
10,marx dormoy,2.425928e+06,0.039983,5344.315789,5344.315789,2672.157895,1,12,nan,nan,...,36.82,37.68,43.01,38.88,31.06,21.78,14.49,10.47,8.05,139




Tableau: ligne numéro[13]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-denis-universite,3569990.0,0.042992,8049.368421,8049.368421,4024.684211,1,13,nan,nan,...,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36,74
1,les courtilles,2802689.0,0.033751,8152.789474,8152.789474,4076.394737,1,13,nan,nan,...,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88,122
2,porte de saint-ouen,2710638.0,0.032643,6262.368421,6262.368421,3131.184211,1,13,nan,nan,...,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27,125
3,montparnasse-bienvenue,5101806.0,0.061438,67112.684211,16778.171053,8389.085526,4,4,6,12,...,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49,4
5,porte de clichy,2639248.5,0.031783,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,place clichy,2580966.0,0.031081,10922.736842,5461.368421,2730.684211,2,2,13,nan,...,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26,33
7,porte de vanves,3395358.0,0.040889,7711.315789,7711.315789,3855.657895,1,13,nan,nan,...,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11,87
8,pernety,2173567.0,0.026175,4404.263158,4404.263158,2202.131579,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
9,carrefour pleyel,1535528.0,0.018492,3933.789474,3933.789474,1966.894737,1,13,nan,nan,...,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78,230
10,gabriel peri,3790573.0,0.045648,9837.842105,9837.842105,4918.921053,1,13,nan,nan,...,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40,64




Tableau: ligne numéro[14]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,gare de lyon,9546825.0,0.172042,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
2,bercy,1942106.0,0.034998,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61
4,madeleine,1776976.0,0.032023,12904.421053,4301.473684,2150.736842,3,8,12,14,...,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10,27
5,porte de clichy,2639248.5,0.047561,12615.105263,6307.552632,3153.776316,2,13,14,nan,...,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66,29
6,cour saint-emilion,2985122.0,0.053794,3173.444444,3173.444444,1586.722222,1,14,nan,nan,...,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76,110
7,bibliotheque,11104474.0,0.200112,6088.000000,6088.000000,3044.000000,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
8,chatelet,1670158.8,0.030098,87779.526316,17555.905263,8777.952632,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
9,olympiades,5214595.0,0.093971,12997.142857,12997.142857,6498.571429,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
10,saint-lazare,6625676.8,0.119400,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
12,mairie de saint-ouen,2415405.0,0.043528,12017.736842,6008.868421,3004.434211,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39


In [18]:
# Créer le dictionnaire pour les nouvelles variables
tableaux_pour_variables = {}

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes spécifiées
    columns_to_drop = ['Rang', 'Trafic', 'nb_correspondance', 'fréquentation_totale_ligne', 
                       'Part du trafic de la station par rapport à toute la ligne',
                       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station']

    # Créer un nouveau tableau sans les colonnes spécifiées
    new_table = current_table.drop(columns=columns_to_drop)

    # Ajouter le nouveau tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = new_table

    # Afficher le tableau mis à jour
    print(f"Tableau 'ligne numéro[{k_str}]' sans les colonnes spécifiées :")
    display(tableaux[f'ligne numéro[{k_str}]'])


Tableau 'ligne numéro[1]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,gare de lyon,22012.649123,11006.324561,1,14,,nan,nan,756681.396934,2.16,...,27.26,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80
2,belleville,8305.815789,4152.907895,2,11,nan,nan,nan,756681.396934,7.73,...,35.54,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97
3,saint-mande-tourelle,8727.684211,4363.842105,1,nan,nan,nan,nan,756681.396934,3.59,...,34.11,35.69,37.54,44.98,47.84,34.26,19.04,10.38,8.05,5.99
4,bercy,4511.973684,2255.986842,6,14,nan,nan,nan,756681.396934,NaN,...,28.84,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09
6,saint-denis-universite,8049.368421,4024.684211,13,nan,nan,nan,nan,756681.396934,1.95,...,28.65,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,michel-ange-auteuil,1449.815789,724.907895,9,10,nan,nan,nan,756681.396934,6.29,...,34.92,37.63,40.86,43.35,40.46,35.29,21.50,11.78,11.57,9.71
124,notre-dame-des-champs,1661.578947,830.789474,12,nan,nan,nan,nan,756681.396934,7.29,...,29.43,36.70,46.97,51.64,51.20,38.02,25.48,14.76,14.98,12.61
125,malakoff-rue etienne dolet,3242.105263,1621.052632,13,nan,nan,nan,nan,756681.396934,2.40,...,33.32,33.51,34.59,43.10,41.24,28.00,15.65,8.94,6.14,4.88
126,michel-ange-molitor,1290.657895,645.328947,9,10,nan,nan,nan,756681.396934,6.79,...,32.38,37.02,39.89,42.74,38.67,35.84,19.83,11.89,10.72,13.92


Tableau 'ligne numéro[2]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,belleville,8305.815789,4152.907895,2,11,nan,nan,nan,297028.650658,7.73,...,35.54,38.84,39.88,41.62,37.49,32.01,21.30,14.72,13.35,11.97
1,colonel fabien,6090.315789,3045.157895,2,nan,nan,nan,nan,297028.650658,7.43,...,32.24,35.77,39.02,45.10,41.33,34.81,22.89,15.25,12.20,10.86
2,menilmontant,5824.315789,2912.157895,2,nan,nan,nan,nan,297028.650658,13.64,...,30.90,32.69,34.55,37.85,37.05,33.65,24.10,18.15,17.53,17.79
3,blanche,4236.421053,2118.210526,2,nan,nan,nan,nan,297028.650658,16.80,...,29.47,34.19,37.46,41.29,41.39,36.88,27.88,21.15,27.09,24.68
4,rue du bac,2642.842105,1321.421053,12,nan,nan,nan,nan,297028.650658,7.98,...,33.95,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93
5,philippe auguste,2473.842105,1236.921053,2,nan,nan,nan,nan,297028.650658,5.48,...,33.32,36.68,41.25,49.12,42.83,32.67,21.55,13.32,12.08,9.82
6,monceau,2529.789474,1264.894737,2,nan,nan,nan,nan,297028.650658,2.91,...,31.56,39.02,48.87,62.67,65.78,47.89,22.54,11.59,7.52,5.00
7,rennes,1546.052632,773.026316,12,nan,nan,nan,nan,297028.650658,4.45,...,38.12,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72
8,montparnasse-bienvenue,16778.171053,8389.085526,4,6,12,13.0,nan,297028.650658,5.71,...,22.86,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49
10,madeleine,4301.473684,2150.736842,8,12,14,nan,nan,297028.650658,4.97,...,31.86,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10


Tableau 'ligne numéro[3]' sans les colonnes spécifiées :


...

,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,rue du bac,2642.842105,1321.421053,12,nan,nan,nan,nan,152539.641886,7.98,...,33.95,40.07,50.13,57.77,57.82,48.87,23.13,15.46,12.09,11.93
1,rennes,1546.052632,773.026316,12,nan,nan,nan,nan,152539.641886,4.45,...,38.12,44.08,52.01,58.20,56.48,44.05,23.63,10.44,10.24,8.72
2,montparnasse-bienvenue,16778.171053,8389.085526,4,6,12,13.0,nan,152539.641886,5.71,...,22.86,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49
4,madeleine,4301.473684,2150.736842,8,12,14,nan,nan,152539.641886,4.97,...,31.86,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10
5,sevres-babylone,3270.052632,1635.026316,10,12,nan,nan,nan,152539.641886,4.71,...,31.21,39.67,51.13,59.22,66.54,57.85,35.27,12.25,9.71,8.96
6,mairie d'issy,6616.526316,3308.263158,12,nan,nan,nan,nan,152539.641886,2.55,...,33.94,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20
7,notre-dame-de-lorette,3949.578947,1974.789474,12,nan,nan,nan,nan,152539.641886,10.62,...,31.34,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33
8,porte de versailles,15023.947368,7511.973684,12,nan,nan,nan,nan,152539.641886,12.16,...,28.53,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49
9,vaugirard,4768.736842,2384.368421,12,nan,nan,nan,nan,152539.641886,4.38,...,35.49,37.27,41.28,44.22,41.04,34.89,19.16,10.52,9.18,7.66
10,marx dormoy,5344.315789,2672.157895,12,nan,nan,nan,nan,152539.641886,4.79,...,33.11,36.82,37.68,43.01,38.88,31.06,21.78,14.49,10.47,8.05


Tableau 'ligne numéro[13]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,saint-denis-universite,8049.368421,4024.684211,13,nan,nan,nan,nan,196412.634211,1.95,...,28.65,30.33,28.90,31.51,27.77,18.64,13.60,8.89,6.17,4.36
1,les courtilles,8152.789474,4076.394737,13,nan,nan,nan,nan,196412.634211,2.39,...,30.01,30.03,31.76,35.24,31.76,23.70,16.18,10.77,7.34,4.88
2,porte de saint-ouen,6262.368421,3131.184211,13,nan,nan,nan,nan,196412.634211,5.48,...,32.40,34.06,38.01,42.89,40.38,34.53,23.47,16.62,12.24,9.27
3,montparnasse-bienvenue,16778.171053,8389.085526,4,6,12,13.0,nan,196412.634211,5.71,...,22.86,26.18,36.84,55.50,62.72,49.65,31.05,21.54,16.83,13.49
5,porte de clichy,6307.552632,3153.776316,13,14,nan,nan,nan,196412.634211,9.95,...,61.47,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66
6,place clichy,5461.368421,2730.684211,2,13,nan,nan,nan,196412.634211,13.84,...,30.26,34.37,37.23,41.68,40.48,38.04,26.06,20.67,21.62,21.26
7,porte de vanves,7711.315789,3855.657895,13,nan,nan,nan,nan,196412.634211,4.61,...,31.46,32.51,35.64,41.18,39.19,30.95,21.34,13.95,10.18,8.11
8,pernety,4404.263158,2202.131579,13,nan,nan,nan,nan,196412.634211,5.99,...,33.18,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96
9,carrefour pleyel,3933.789474,1966.894737,13,nan,nan,nan,nan,196412.634211,3.40,...,30.52,33.97,48.79,57.68,45.30,28.87,16.22,10.19,8.39,6.78
10,gabriel peri,9837.842105,4918.921053,13,nan,nan,nan,nan,196412.634211,3.24,...,31.64,31.95,34.54,38.88,38.49,29.48,18.40,11.34,7.23,5.40


Tableau 'ligne numéro[14]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,gare de lyon,22012.649123,11006.324561,1,14,,nan,nan,131269.920635,2.16,...,27.26,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80
2,bercy,4511.973684,2255.986842,6,14,nan,nan,nan,131269.920635,NaN,...,28.84,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09
4,madeleine,4301.473684,2150.736842,8,12,14,nan,nan,131269.920635,4.97,...,31.86,43.85,53.91,65.73,71.49,60.81,23.76,11.64,11.65,10.10
5,porte de clichy,6307.552632,3153.776316,13,14,nan,nan,nan,131269.920635,9.95,...,61.47,67.67,75.58,92.43,88.85,71.86,46.94,32.62,24.38,18.66
6,cour saint-emilion,3173.444444,1586.722222,14,nan,nan,nan,nan,131269.920635,11.21,...,28.48,36.59,49.66,61.54,60.91,45.62,29.16,25.69,15.63,12.76
7,bibliotheque,6088.000000,3044.000000,14,nan,nan,nan,nan,131269.920635,6.46,...,30.38,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76
8,chatelet,17555.905263,8777.952632,1,4,7,11.0,14.0,131269.920635,30.17,...,55.16,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60
9,olympiades,12997.142857,6498.571429,14,nan,nan,nan,nan,131269.920635,2.74,...,35.48,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47
10,saint-lazare,25432.673684,12716.336842,3,9,12,13.0,14.0,131269.920635,8.62,...,23.72,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86
12,mairie de saint-ouen,6008.868421,3004.434211,13,14,nan,nan,nan,131269.920635,7.75,...,62.26,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66


In [19]:
for k in numéro:
    k_str = str(k)
    tableaux[f'ligne numéro[{k_str}]'].to_csv(f'ligne_numero_{k_str}.csv', index=False) 

In [20]:
def affluence_hors_jo(horaire, station, ligne):
    # Obtenir le tableau correspondant à la ligne
    line_table = tableaux[f'ligne numéro[{ligne}]']

    # Extraire l'heure de l'horaire entré par l'utilisateur
    hour, minute = map(int, horaire.split(':'))

    # Construire la chaîne de plage horaire correspondante
    plage_horaire = f"{hour:02d}:00-{(hour + 1) % 24:02d}:00"

    # Vérifier si la plage horaire se trouve dans les colonnes du tableau
    if plage_horaire not in line_table.columns:
        raise ValueError(f"Plage horaire non trouvée dans le tableau: {plage_horaire}")

    # Récupérer le nombre à la station et à la plage horaire donnée
    affluence_horaire = line_table[line_table['Station'] == station][plage_horaire].iloc[0]

    # Récupérer l'indice de la plage horaire suivante
    next_time_range_index = line_table.columns.get_loc(plage_horaire) + 1
    next_time_range_index = next_time_range_index % len(line_table.columns)  # Assurer la circularité

    # Récupérer la plage horaire suivante
    plage_horaire_suivante = line_table.columns[next_time_range_index]

    # Récupérer le nombre à la plage horaire suivante
    affluence_suivante = line_table[line_table['Station'] == station][plage_horaire_suivante].iloc[0]

    # Récupérer la moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction à la station
    moyenne_ligne = line_table[line_table['Station'] == station]['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'].iloc[0]

    return round(affluence_horaire), round(affluence_suivante), round(moyenne_ligne)

# Exemple d'utilisation
horaire_info = "01:35"
station_info = "bastille"
ligne_info = "1"

try:
    affluence_horaire, affluence_suivante, affluence_ligne = affluence_hors_jo(horaire_info, station_info, ligne_info)
    print(f"Affluence à {horaire_info} à la station {station_info}, sur la ligne numéro {ligne_info} : {affluence_horaire}")
    print(f"Affluence à la plage horaire suivante à la station {station_info}, sur la ligne numéro {ligne_info} : {affluence_suivante}")
    print(f"Affluence sur la ligne {ligne_info} : {affluence_ligne}")
except ValueError as e:
    print(e)


Affluence à 01:35 à la station bastille, sur la ligne numéro 1 : 8
Affluence à la plage horaire suivante à la station bastille, sur la ligne numéro 1 : 0
Affluence sur la ligne 1 : 2633
